In [1]:
import os.path
!pip freeze | grep scikit-learn

scikit-learn==1.3.2


In [2]:
!python -V

Python 3.8.10


In [15]:
import pickle
import pandas as pd
import numpy as np
import os
import pyarrow

In [4]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/home/kirill/ML OPS/MLOps_training/.venv/lib/python3.8/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/kirill/ML OPS/MLOps_training/.venv/lib/python3.8/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [6]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

In [7]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [22]:
standart_deviation = np.std(y_pred)
print(standart_deviation)

14.203865642696083


In [18]:
year = 2023
month = 3
output_file = "./output.parquet"
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['predicted_duration'] = y_pred

df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

print((os.path.getsize(output_file) / 1024) / 1024)

65.46199798583984


In [21]:
!jupyter nbconvert --to script homework_4.ipynb

[NbConvertApp] Converting notebook ./homework_4.ipynb to script
[NbConvertApp] Writing 1552 bytes to homework_4.py


In [27]:
!pip freeze | grep scikit-learn
!pip freeze | grep pickle

scikit-learn==1.3.2
cloudpickle==3.1.1
pickleshare==0.7.5


In [ ]:
!pipenv install scikit-learn==1.3.2 pickle pandas munpy
!pipenv shell

In [34]:
!python3 homework_4.py --year 2023 --month 04

/home/kirill/ML OPS/MLOps_training/.venv/lib/python3.8/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/kirill/ML OPS/MLOps_training/.venv/lib/python3.8/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
14.292282936862449


In [50]:
from getpass import getpass
password = getpass("Enter sudo password: ")

!echo {password} | sudo -S docker build -t homework_4 .
!echo {password} | sudo -S docker run --rm homework_4

[sudo] password for kirill: [+] Building 0.0s (0/1)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.0s
[+] Building 0.2s (1/2)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 271B                                       0.0s
 => [internal] load metadata for docker.io/agrigorev/zoomcamp-model:mlops  0.2s
[+] Building 0.3s (1/2)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 271B                                       0.0s
 => [internal] load metadata for docker.io/agrigorev/zoomcamp-model:mlops  0.3s
[+] Building 0.5s (1/2)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => trans